In [1]:
!pip install datasets transformers peft trl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.7/320.7 kB 5.5 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.2/310.2 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 71.4 MB/s eta 0:00:00:00:01:01m
  Attempting uninstall: transformers
    Found existing installation: transformers 4.45.1
    Uninstalling transformers-4.45.1:
      Successfully uninstalled transformers-4.45.1


In [2]:
!pip install -U bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 12.4 MB/s eta 0:00:0000:0100:01


In [3]:
import pandas as pd
from datasets import Dataset
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import get_peft_model, LoraConfig, TaskType
from transformers import TrainingArguments
from trl import SFTTrainer


In [4]:
# Charger le dataset
dataset = pd.read_csv("https://raw.githubusercontent.com/xashru/cti-bench/main/data/cti-mcq.tsv", sep='\t')
dataset.head()

,URL,Question,Option A,Option B,Option C,Option D,Prompt,GT
0,https://attack.mitre.org/techniques/T1548/,Which of the following mitigations involves pr...,Audit,Execution Prevention,Operating System Configuration,User Account Control,You are given a multiple-choice question (MCQ)...,B
1,https://attack.mitre.org/techniques/T1548/,Which data source is recommended for monitorin...,Command,File,Process,User Account,You are given a multiple-choice question (MCQ)...,D
2,https://attack.mitre.org/techniques/T1548/,What does mitigation ID M1028 suggest to preve...,Limiting privileges of cloud accounts,Preventing unsigned applications from running,Minimizing applications with setuid or setgid ...,Enforcing the highest UAC level,You are given a multiple-choice question (MCQ)...,C
3,https://attack.mitre.org/techniques/T1548/,Which process creation is an indicator of pote...,C:\Windows\System32\services.exe,C:\Windows\System32\cmd.exe,C:\Windows\System32\rundll32.exe,C:\Windows\System32\notepad.exe,You are given a multiple-choice question (MCQ)...,B
4,https://attack.mitre.org/techniques/T1548/,"In a Linux environment, what is recommended to...",Monitor Windows Registry Key Modification,Monitor OS API Execution,Monitor file metadata for setuid or setgid bit...,Audit process metadata changes,You are given a multiple-choice question (MCQ)...,C


In [5]:
def format_prompt(row):
    prompt = (f"Question: {row['Question']}\n"
              f"Options:\n"
              f"A) {row['Option A']}\n"
              f"B) {row['Option B']}\n"
              f"C) {row['Option C']}\n"
              f"D) {row['Option D']}\n\n"
              f"Veuillez sélectionner la meilleure réponse.\n")
             
    return prompt

In [6]:
# Préparer l'ensemble complet de données
dataset['Prompt'] = dataset.apply(format_prompt, axis=1)
formatted_dataset = Dataset.from_pandas(dataset)
formatted_dataset

Dataset({
    features: ['URL', 'Question', 'Option A', 'Option B', 'Option C', 'Option D', 'Prompt', 'GT'],
    num_rows: 2500
})

In [7]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

# Configurer pour charger en 8-bit
quantization_config = BitsAndBytesConfig(load_in_8bit=True)

# Charger le tokenizer et le modèle avec quantization
model_name = "unsloth/llama-2-7b"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=quantization_config,  # Utiliser la config de quantization
    device_map="auto"  # Répartition automatique sur GPU/CPU
)


tokenizer_config.json:   0%|          | 0.00/948 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/438 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/752 [00:00<?, ?B/s]

...

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/3.59G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/183 [00:00<?, ?B/s]

In [8]:

# Configurer Lora pour le fine-tuning
lora_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    r=4,
    lora_alpha=8,
    lora_dropout=0.2,
)

In [9]:
model = get_peft_model(model, lora_config)

In [10]:
# Configurer les arguments d'entraînement
training_args = TrainingArguments(
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    max_steps=200,
    learning_rate=2e-5,
    output_dir="outputs",
    logging_dir="logs",
    report_to="tensorboard",
    fp16=True,
)

In [11]:
# Initialiser le trainer
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=formatted_dataset,
    dataset_text_field="Prompt",
    args=training_args,
)

/opt/conda/lib/python3.10/site-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field. Will not be supported from version '0.13.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/opt/conda/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:309: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:328: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(


Map:   0%|          | 0/2500 [00:00<?, ? examples/s]

/opt/conda/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:403: UserWarning: You passed a processing_class with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `processing_class.padding_side = 'right'` to your code.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)
max_steps is given, it will override any value given in num_train_epochs


In [12]:
# Entraîner le modèle 
trainer.train()

Step,Training Loss


TrainOutput(global_step=200, training_loss=2.0995582580566405, metrics={'train_runtime': 985.0583, 'train_samples_per_second': 1.624, 'train_steps_per_second': 0.203, 'total_flos': 6469505688059904.0, 'train_loss': 2.0995582580566405, 'epoch': 0.64})

In [13]:

model.save_pretrained("fine_tuning")
print("Modèle enregistré ")

Modèle enregistré 


In [14]:

# Fonction de formatage du prompt
def format_prompt_val(row):
    question_prompt = f"Q: {row['Question']}\nA) {row['Option A']}\nB) {row['Option B']}\nC) {row['Option C']}\nD) {row['Option D']}\nPlease respond with only the letter of your answer (A, B, C, or D):"
    return question_prompt


In [15]:

# Charger le dataset de validation
validation_dataset = pd.read_csv("/kaggle/input/cti-mcq/CTI_MCQ (3).tsv", sep='\t', on_bad_lines='skip')
validation_dataset['Prompt'] = validation_dataset.apply(format_prompt_val, axis=1)


In [16]:
# Transformer le dataframe en un Dataset Hugging Face
formatted_validation_dataset = Dataset.from_pandas(validation_dataset)

In [17]:
# Fonction d'évaluation pour générer les réponses dans le format souhaité
def evaluate_model(model, tokenizer, dataset, output_file, batch_size=8):
    model.eval()
    total = len(dataset)  

    with open(output_file, 'w') as f:
        
        f.write("LLAMA2-7B\tGT\n")

        for i in range(0, total, batch_size):
            batch = dataset.select(range(i, min(i + batch_size, total)))
            prompts = [entry['Prompt'] for entry in batch]

            # Préparer les entrées pour le modèle
            inputs = tokenizer(prompts, return_tensors="pt", padding=True, truncation=True)

            try:
                with torch.no_grad():
                    outputs = model.generate(**inputs, max_new_tokens=500) 

                generated_texts = tokenizer.batch_decode(outputs, skip_special_tokens=True)

                for j, generated_text in enumerate(generated_texts):
                    # Extraction de la lettre de réponse
                    answer = generated_text.strip().split()[0] if generated_text else ""  

                    # Vérifier si la réponse est une lettre valide
                    if answer not in ['A', 'B', 'C', 'D']:
                        answer = ''  

                    correct_answer = batch[j]['GT'].strip()  # Réponse correcte

                    # Écrire la réponse générée et la réponse correcte dans le fichier
                    f.write(f"{answer}\t{correct_answer}\n")  

            except RuntimeError as e:
                continue  


In [ ]:
# Appeler la fonction pour créer le fichier de sortie
evaluate_model(model, tokenizer, formatted_validation_dataset, "CTI_reponse.txt", batch_size=8, num_questions=50)

In [62]:
file_path = '/kaggle/working/CTI_reponse.txt'

# Charger le fichier en sautant la première ligne
results = pd.read_csv(file_path, sep='\s+', skiprows=1, header=None)

# Renommer les colonnes
results.columns = ["LLAMA2-7B", "GT"]

# Calculer la précision en comparant les colonnes
accuracy = (results["LLAMA2-7B"] == results["GT"]).mean()

# Afficher la précision
print(f'Accuracy: {accuracy:.2%}')

Accuracy: 92.99%
